<a href="https://colab.research.google.com/github/phdkiran/mkmoney/blob/master/Xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import os
from google.colab import drive, html, auth, files
drive.mount('/content/gdrive')
# os.mkdir('/content/gdrive/My Drive/kaggle/xray/')
os.chdir('/content/gdrive/My Drive/kaggle/xray/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [47]:
'''
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
'''

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
kaggle.json


files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle


!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip
!unzip chest_xray.zip

!rm -rf chest-xray-pneumonia.zip
!rm -rf chest_xray.zip
!rm -rf __MACOSX
# !rm -rf kaggle.json

## Prepare and preprocess data

In [0]:
train_dir = 'chest_xray/train/'
validation_dir = 'chest_xray/val/'
test_dir = 'chest_xray/test/'

In [0]:
from tensorflow.keras.applications import inception_v3 as model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, layers
from tensorflow.keras.optimizers import Adam

# bottle_neck preprocess function
bottle_neck_input = model.preprocess_input

train_datagen = ImageDataGenerator(
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=bottle_neck_input)

valid_datagen = ImageDataGenerator(preprocessing_function=bottle_neck_input)

In [4]:
# Feed directories
train_gen = train_datagen.flow_from_directory(directory=train_dir, batch_size=16, color_mode='rgb', class_mode='binary', target_size=(256,256))
val_gen = train_datagen.flow_from_directory(directory=validation_dir, shuffle=False, batch_size=16, color_mode='rgb', class_mode='binary', target_size=(256,256))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
base_model = model.InceptionV3(include_top=False, weights='imagenet')
for layer in base_model.layers:
  layer.trainable = False

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
87916544/87910968 [==============================] - 8s 0us/step


In [0]:
# !pip install -U keras_applications

In [0]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
preds = layers.Dense(2, activation='softmax')(x) #Final predictions

optim = Adam(lr=0.001)
final_model = Model(base_model.input, preds)
final_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optim,
              metrics=['accuracy'])


In [8]:
history = final_model.fit_generator(generator=train_gen,
                              epochs=3,
                              validation_data=val_gen)

Epoch 1/3
326/326 [==============================] - 3057s 9s/step - loss: 0.2985 - acc: 0.8744 - val_loss: 2.1059 - val_acc: 0.5000
Epoch 2/3
326/326 [==============================] - 149s 458ms/step - loss: 0.2734 - acc: 0.8861 - val_loss: 2.3651 - val_acc: 0.5000
Epoch 3/3
326/326 [==============================] - 154s 472ms/step - loss: 0.2341 - acc: 0.9049 - val_loss: 1.2740 - val_acc: 0.5000


In [0]:
final_model.save('./model.h5')
final_model.save_weights('./model_weights.h5')

In [12]:
open('./model.json', 'w').write(final_model.to_json())

158084

In [51]:
print(final_model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [45]:
import json
print(json.dumps(json.loads(final_model.to_json()), indent=2))
!ls -lh model*

{
  "class_name": "Model",
  "config": {
    "name": "model",
    "layers": [
      {
        "name": "input_1",
        "class_name": "InputLayer",
        "config": {
          "batch_input_shape": [
            null,
            null,
            null,
            3
          ],
          "dtype": "float32",
          "sparse": false,
          "name": "input_1"
        },
        "inbound_nodes": []
      },
      {
        "name": "conv2d",
        "class_name": "Conv2D",
        "config": {
          "name": "conv2d",
          "trainable": false,
          "dtype": "float32",
          "filters": 32,
          "kernel_size": [
            3,
            3
          ],
          "strides": [
            2,
            2
          ],
          "padding": "valid",
          "data_format": "channels_last",
          "dilation_rate": [
            1,
            1
          ],
          "activation": "linear",
          "use_bias": false,
          "kernel_initializer": {
           

In [20]:
test_datagen = ImageDataGenerator(preprocessing_function=bottle_neck_input)

test_gen = test_datagen.flow_from_directory(directory=test_dir, shuffle=False, batch_size=1, color_mode='rgb', class_mode='binary', target_size=(256,256))


Found 624 images belonging to 2 classes.


In [21]:

test_gen.reset()
predictions = final_model.predict(test_gen, verbose=1)

624/624 [==============================] - 346s 554ms/step


In [43]:
import numpy as np
import pandas as pd
d = train_gen.class_indices
d = {v:k for k,v in d.items()}
df = pd.DataFrame(np.argmax(predictions, axis=1).reshape(-1, 1)).applymap(d.get)
df.to_csv('./predictions.csv')
df.head()

,0
0,PNEUMONIA
1,NORMAL
2,NORMAL
3,NORMAL
4,NORMAL


In [0]:
# ! kaggle competitions submit -c paultimothymooney/chest-xray-pneumonia -f './predictions.csv' -m "initial submission"

In [22]:
'''
References: 
https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
https://deepsense.ai/keras-vs-pytorch-avp-transfer-learning/
'''

'\nReferences: \nhttps://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720\nhttps://deepsense.ai/keras-vs-pytorch-avp-transfer-learning/\n'